In [ ]:
# REPL stands for Read, Evaluate, Print, Loop. 
# The REPL is how you interact with the Python Interpreter. 
# Unlike running a file containing Python code, in the REPL you can type commands 
# and instantly see the output printed out.

# https://python.langchain.com/api_reference/langchain/agents/langchain.agents.tool_calling_agent.base.create_tool_calling_agent.html#create-tool-calling-agent
# Agent with memory https://python.langchain.com/docs/how_to/agent_executor/
# Based on https://notes.kodekloud.com/docs/LangChain/Building-Agents/Adding-Pyton-REPL-to-Tool
from langchain_core.prompts.chat import ChatPromptTemplate

from langchain_openai import ChatOpenAI
from langchain_experimental.tools import PythonREPLTool


from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor


In [65]:
from langchain.agents import tool

# Tool to get flight statuss
@tool
def countries_independence_day(region: str) -> str:
    """Gets countries's independence day."""
    return (
        f"The countries date of independence of {region} in markdown format are as follows: \n"
        "| Country      | Independence Date | Notes |\n"
        "|--------------|-------------------|-------|\n"
        "| **Paraguay** | 14/05/1811 | From Spain |\n"
        "| **Venezuela** | 05/07/1811 | First declaration; definitive independence in 1821 |\n"
        "| **Argentina** | 09/07/1816 | From Spain |\n"
        "| **Chile** | 12/02/1818 | From Spain |\n"
        "| **Colombia** | 20/07/1810 | Declaration from Spain (consolidated 07/08/1819) |\n"
        "| **Ecuador** | 24/05/1822 | From Spain (Battle of Pichincha) |\n"
        "| **Peru** | 28/07/1821 | Proclamation by San Martín; final victory 09/12/1824 |\n"
        "| **Bolivia** | 06/08/1825 | From Spain |\n"
        "| **Uruguay** | 25/08/1825 | From Brazil; recognized 27/08/1828 |\n"
        "| **Brazil** | 07/09/1822 | From Portugal |\n"
        "| **Guyana** | 26/05/1966 | From the United Kingdom |\n"
        "| **Suriname** | 25/11/1975 | From the Netherlands |\n"
    )
countries_independence_day

StructuredTool(name='countries_independence_day', description="Gets countries's independence day.", args_schema=<class 'langchain_core.utils.pydantic.countries_independence_day'>, func=<function countries_independence_day at 0x11ba8b1a0>)

In [69]:
# Initialize chat model, prompt and output parser
from langchain.prompts import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Think step by step before responding."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

In [70]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [71]:

python_repl = PythonREPLTool()

llm = ChatOpenAI(verbose=True, temperature=0)
tools = [countries_independence_day, python_repl]

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [72]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [74]:
response = agent_with_chat_history.invoke(
    {
        "input": "Give the countries of South America that became independent after 1900.",
     },
    config={"configurable": {"session_id": "mysession"}}
)
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `countries_independence_day` with `{'region': 'South America'}`


The countries date of independence of South America in markdown format are as follows: 
| Country      | Independence Date | Notes |
|--------------|-------------------|-------|
| **Paraguay** | 14/05/1811 | From Spain |
| **Venezuela** | 05/07/1811 | First declaration; definitive independence in 1821 |
| **Argentina** | 09/07/1816 | From Spain |
| **Chile** | 12/02/1818 | From Spain |
| **Colombia** | 20/07/1810 | Declaration from Spain (consolidated 07/08/1819) |
| **Ecuador** | 24/05/1822 | From Spain (Battle of Pichincha) |
| **Peru** | 28/07/1821 | Proclamation by San Martín; final victory 09/12/1824 |
| **Bolivia** | 06/08/1825 | From Spain |
| **Uruguay** | 25/08/1825 | From Brazil; recognized 27/08/1828 |
| **Brazil** | 07/09/1822 | From Portugal |
| **Guyana** | 26/05/1966 | From the United Kingdom |
| **Suriname** | 25/11/1975 | From the Netherlands |
The count

In [75]:
response = agent_with_chat_history.invoke(
    {
        "input": "calculate the difference in days between them",
     },
    config={"configurable": {"session_id": "mysession"}}
)
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `countries_independence_day` with `{'region': 'Guyana'}`


The countries date of independence of Guyana in markdown format are as follows: 
| Country      | Independence Date | Notes |
|--------------|-------------------|-------|
| **Paraguay** | 14/05/1811 | From Spain |
| **Venezuela** | 05/07/1811 | First declaration; definitive independence in 1821 |
| **Argentina** | 09/07/1816 | From Spain |
| **Chile** | 12/02/1818 | From Spain |
| **Colombia** | 20/07/1810 | Declaration from Spain (consolidated 07/08/1819) |
| **Ecuador** | 24/05/1822 | From Spain (Battle of Pichincha) |
| **Peru** | 28/07/1821 | Proclamation by San Martín; final victory 09/12/1824 |
| **Bolivia** | 06/08/1825 | From Spain |
| **Uruguay** | 25/08/1825 | From Brazil; recognized 27/08/1828 |
| **Brazil** | 07/09/1822 | From Portugal |
| **Guyana** | 26/05/1966 | From the United Kingdom |
| **Suriname** | 25/11/1975 | From the Netherlands |

Invoking: `countries_i

Python REPL can execute arbitrary code. Use with caution.



Invoking: `Python_REPL` with `{'query': "from datetime import datetime\n\ndate1 = datetime.strptime('26/05/1966', '%d/%m/%Y')\ndate2 = datetime.strptime('25/11/1975', '%d/%m/%Y')\n\ndays_difference = (date2 - date1).days\nprint(days_difference)"}`
responded: The independence day of Guyana is 26/05/1966, and the independence day of Suriname is 25/11/1975. Let's calculate the difference in days between these two dates.

3470


0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


The difference in days between the independence days of Guyana and Suriname is 3,470 days.

> Finished chain.
The difference in days between the independence days of Guyana and Suriname is 3,470 days.
